<a href="https://colab.research.google.com/github/Samir-atra/Expenses_tracker/blob/main/project/utils/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import pathlib
import IPython
import sys
from sklearn.model_selection import train_test_split
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_dir = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/Tobacco3482-jpg/')

# def load_data(dir):
#     img_data_array = []
#     class_name = []
#     class_num = 0
#     class_dict = dict()

#     for dir1 in os.listdir(dir):
#         # print("this is dir1", dir1)
#         for filey in os.listdir(os.path.join(dir, dir1)):
#             image_path = os.path.join(dir, dir1, filey)
#             image = cv2.imread(image_path, cv2.COLOR_BGR2RGB)
#             if image is None:
#                 print("beedoo")
#                 continue
#             image = cv2.resize(image, (300, 300))
#             image = np.array(image)
#             image = image.astype("float32")
#             # image /= 255
#             if len(image.shape) < 3:
#                 continue
#             else:
#                 img_data_array.append(image)
#                 class_name.append(class_num)
#         print(class_num, dir1)
#         class_dict.update({class_num: dir1})
#         class_num +=1

#     return img_data_array, class_name, class_dict




# Split data into training and testing sets
# img_data_array, class_name, class_dict = load_data(data_dir)
# labels = tf.keras.utils.to_categorical(class_name)
# print(labels)
# x_train, x_test, y_train, y_test = train_test_split(np.array(img_data_array), np.array(class_name), test_size=0.3)

# data_path_test = pathlib.Path('/content/drive/MyDrive/archiveX3/Testing/')
# AUTOTUNE=tf.data.AUTOTUNE

dataset_path = tf.keras.utils.image_dataset_from_directory(        # Training dataset
    data_dir,
    labels= 'inferred',
    # validation_split=0.2,
    # subset='training',
    seed= 1,
    batch_size=5,
    image_size=(300,300),
    color_mode="rgb",
    shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

# dataset_path_val = tf.keras.utils.image_dataset_from_directory(      #Validation dataset
#     data_path,
#     labels= 'inferred',
#     validation_split=0.2,
#     subset='validation',
#     seed= 2,
#     batch_size=5,
#     image_size=(299, 299),
#     color_mode="rgb",
#     shuffle=True)#.cache().prefetch(buffer_size=AUTOTUNE)

Found 4366 files belonging to 12 classes.
Using 3493 files for training.


In [ ]:
base_model = tf.keras.applications.inception_v3.InceptionV3(
    input_shape = (300, 300, 3),
    include_top = False,
    weights = "imagenet"
)

In [ ]:
saving_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/expenses_tracker_classifier95%test.h5')

base_model.save(saving_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
base_model = tf.keras.models.load_model(saving_path)                 #Loading base_model

base_model.trainable = False                                        # Setting the model as non-trainable

rescaling = tf.keras.Sequential([
  tf.keras.layers.Rescaling(scale=1 / 127.5, offset=-1)             # Rescaling to (1, -1) range required for inceptionV3 model
])
augmentation = tf.keras.Sequential([                                # Applying augmentations o the images
   tf.keras.layers.RandomFlip("horizontal"),
   tf.keras.layers.RandomRotation(0.1)
])


inputs = tf.keras.Input(shape=(300, 300, 3))
# x = augmentation(inputs)
x = rescaling(inputs)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(12, activation='softmax')(x)                   # "softmax" in the final layer for decision making
model = tf.keras.Model(inputs, outputs)


model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
    run_eagerly = True)

checkpoint_filepath = '/content/drive/MyDrive/expenses_tracker_classifier/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='accuracy',
    mode='max',
    save_best_only=True)

model.fit(                                                               # fitting the whole model for non-trainable base
    dataset_path,
    epochs=20,
    callbacks=[model_checkpoint_callback])


# model.load_weights(checkpoint_filepath)
# model.evaluate(
#     x=x_test,
#     y=y_test,)

full_model_saving_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/expenses_tracker_model.h5')

tf.keras.models.save_model(model,                                     # saving the fully trained model
                           full_model_saving_path,
                           overwrite=True,
                           save_format='tf'
                           )

Epoch 1/30
699/699 [==============================] - 725s 1s/step - loss: 1.2913 - accuracy: 0.5843
Epoch 2/30
699/699 [==============================] - 275s 393ms/step - loss: 0.9152 - accuracy: 0.6954
Epoch 3/30
699/699 [==============================] - 277s 395ms/step - loss: 0.8237 - accuracy: 0.7269
Epoch 4/30
699/699 [==============================] - 284s 404ms/step - loss: 0.7752 - accuracy: 0.7286
Epoch 5/30
699/699 [==============================] - 278s 396ms/step - loss: 0.7361 - accuracy: 0.7504
Epoch 6/30
699/699 [==============================] - 279s 399ms/step - loss: 0.7207 - accuracy: 0.7561
Epoch 7/30
699/699 [==============================] - 280s 399ms/step - loss: 0.6939 - accuracy: 0.7638
Epoch 8/30
699/699 [==============================] - 279s 398ms/step - loss: 0.6750 - accuracy: 0.7670
Epoch 9/30
699/699 [==============================] - 282s 401ms/step - loss: 0.6530 - accuracy: 0.7790
Epoch 10/30
699/699 [==============================] - 1778s 3s/ste

In [ ]:
full_model_saving_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/expenses_tracker_model.h5')
loaded_model = tf.keras.models.load_model(full_model_saving_path, compile=True)
pred_img_path = pathlib.Path('/content/drive/MyDrive/expenses_tracker_classifier/pred_imgs/')
# imagess, labelss, _ = load_data(pred_img_path)


loaded_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0005),        # compiling with low learning rate
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits= True),
    metrics=['accuracy'],
    run_eagerly = False)

# dataset_pathy = tf.keras.utils.image_dataset_from_directory(
#     pred_img_path,
#     labels= 'inferred',
#     seed= 1,
#     batch_size=1,
#     image_size=(300, 300),
#     color_mode="rgb",
#     shuffle=False)


# for image in imagess:
# #   #  for i in range(1):
# #       #  ax = plt.subplot(1, 1, i+1)
#    plt.imshow(image.astype("uint8"))
# # #       #  plt.title(class_names[labels[i]])
# # #       #  plt.axis("off")
#    plt.show()


sample_list = []                                                        # creating a list of sample(s)
for sample, label in load_data(pred_img_path):
for sample, labels in dataset_pathy:
    sample_list.append(sample)
    continue

class_dict = {0:"Report",
1:"Letter",
2:"Memo",
3:"Email",
4:"Note",
5:"Scientific",
6:"Form",
7:"Resume",
8:"ADVE",
9:"News",
10:"Receipts",
11:"Empty_Papers",}

sample_list = np.array(sample_list)
for sample in sample_list:
    # print(sample.shape)
    # sample = tf.expand_dims(sample, axis=0)
    predictions = loaded_model.predict(sample)
    pred = np.argmax(predictions, axis=1)
    print(class_dict.get(pred[0]))


Found 5 files belonging to 1 classes.
1/1 [==============================] - 1s 1s/step
Resume
1/1 [==============================] - 0s 28ms/step
Note
1/1 [==============================] - 0s 28ms/step
ADVE
1/1 [==============================] - 0s 28ms/step
ADVE
1/1 [==============================] - 0s 30ms/step
Scientific
